# **Project Name**    - DeepFER: Facial Emotion Recognition Using Transfer Learning



# Transfer Learning using EfficientNet algorithm

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

train_dir = r"original_images\train"
val_dir = r"original_images\test"

# Load grayscale datasets with small input size
train_ds = image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
    image_size=(48, 48),
    batch_size=32,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    val_dir,
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
    image_size=(48, 48),
    batch_size=32,
    shuffle=False
)

# Data Augmentation block
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# Preprocessing function
def preprocess(image, label):
    image = tf.image.grayscale_to_rgb(image)  # Convert 1 channel → 3 channels
    image = tf.image.resize(image, (224, 224), method='bicubic')  # Best interpolation
    image = data_augmentation(image)  # Apply data augmentation
    image = preprocess_input(image)
    return image, label


# Apply preprocessing
train_data = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Optimize pipeline
train_data = train_data.prefetch(tf.data.AUTOTUNE)
val_data = val_data.prefetch(tf.data.AUTOTUNE)

# EffcientNet model
def create_model():
    base_model = EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base initially

    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(7, activation='softmax', dtype='float32')(x)  # 7 classes

    model = models.Model(inputs, outputs)
    return model


model = create_model()
model.summary()

# Compile the Model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="models/emotion_model.keras",
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=8,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        verbose=1
    )
]

# Training the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=callbacks
)

In [ ]:
# Fine-Tuning Stage (Unfreeze Some Layers)
# Unfreeze base model
base_model = model.layers[1]
base_model.trainable = True

# Optionally freeze first few layers to retain low-level features
for layer in base_model.layers[:200]:
    layer.trainable = False

# Re-compile with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_finetune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)

model.save("final_transfer_learning emotion_recognition_model.keras")
